[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/Mnist-dataset-using-CNN/blob/master/MNIST_dataset_using_CNN.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("drive/MNIST Dataset using CNN", one_hot = True)

In [0]:
os.chdir('/content')
!git clone https://github.com/mixuala/colab_utils
import colab_utils.tboard
ROOT = %pwd
print(ROOT)
LOG_DIR = os.path.join(ROOT, 'gaurav')
print(LOG_DIR)
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

In [0]:
batch_size = 1024
hm_epochs = 10
x = tf.placeholder(dtype = tf.float32, shape = [None, 784])
y = tf.placeholder(dtype = tf.int32, shape = [None, 10])
mode = tf.placeholder(dtype = tf.bool)

In [0]:
def CNN_model(x, mode):
  input_layer = tf.reshape(x, [-1, 28, 28, 1])
  conv1 = tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode)

  logits = tf.layers.dense(inputs=dropout, units=10)

  return logits


In [0]:
def train_model(x):
  prediction = CNN_model(x, mode)
  
  with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
  summary = tf.summary.scalar('total_cost_of_batch', cost)
  
  with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)
  
  merged = tf.summary.merge_all()
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./gaurav',sess.graph)
    k = 0
    for i in range(hm_epochs):
      epoch_loss = 0
      for j in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        o, c, m = sess.run([optimizer, cost, merged], feed_dict = {x : epoch_x, y : epoch_y, mode : True})
        epoch_loss += c
        train_writer.add_summary(m, k)
        k += 1
      print('epoch' , i, 'completed out of ', hm_epochs, 'loss ', epoch_loss)
      
      
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
  
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
  
    print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels, mode : False})) 
    


In [9]:
train_model(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

epoch 0 completed out of  10 loss  22.931325517594814
epoch 1 completed out of  10 loss  4.610106658190489
epoch 2 completed out of  10 loss  3.020946491509676
epoch 3 completed out of  10 loss  2.3322547283023596
epoch 4 completed out of  10 loss  1.9687766525894403
epoch 5 completed out of  10 loss  1.5777212800458074
epoch 6 completed out of  10 loss  1.2601962517946959
epoch 7 completed out of  10 loss  1.043330761604011
epoch 8 completed out of  10 loss  0.9821593384258449
epoch 9 completed out of  10 loss  0.7749365344643593
Accuracy: 0.9914
